In [36]:
import helper
data_dir = "./data/simpsons/moes_tavern_lines.txt"
text = helper.load_data(dir)
print(text[:81])
text = text[81:]

[YEAR DATE 1989] © Twentieth Century Fox Film Corporation. All rights reserved.




In [26]:
view_sentence_range = (0, 10)
import numpy as np
print("Data set stats")
print("The number of words in text: {}".format(len({word:None for word in text.split()})))

scenes = text.split("\n\n")
print("Number of scene: {}".format(len(scenes)))

sentence_count_scene = [scene.count("\n") for scene in scenes]
print("Average Number of sentences in each scene: {}".format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split("\n")]
print("Number of lines: {}".format(len(sentences)))

word_count_sentence = [len(sentence.split()) for sentence in sentences]
print("Average words in sentences: {}".format(np.average(word_count_sentence)))

print()
print("The sentences {} to {}".format(*view_sentence_range))
print("\n".join(text.split("\n")[view_sentence_range[0]:view_sentence_range[1]]))

Data set stats
The number of words in text: 11492
Number of scene: 262
Average Number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average words in sentences: 11.50434578341555

The sentences 0 to 10
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [42]:
import problem_unittests as test
from collections import Counter

def create_lookup_tables(text):
    text_counter = Counter(text)
    text_counter = sorted(text_counter, key=text_counter.get, reverse=True)
    vocab_to_int = {word:idx for idx, word in enumerate(text_counter)}
    int_to_vocab = {idx:word for idx, word in enumerate(text_counter)}
    
    return (vocab_to_int, int_to_vocab)

In [43]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    Tokenize = { '.':'||Period||', ',':'||Comma||', '"':'||Quotation_Mark||', ';':'||Semicolon||',\
                '!':'||Exclamation_mark||', '?':'||Question_mark||', '(':'||Left_Parentheses||', \
                ')':'||Right_Parentheses||', '--':'||Dash||', '\n':'||Return||' }
                
    return Tokenize

In [44]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [45]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [47]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None,None], name="input")
    Targets = tf.placeholder(tf.int32, [None,None], name="targets")
    LearningRate = tf.placeholder(tf.int32, name="learning_rate")
    return (inputs, Targets, LearningRate)

In [49]:
def get_init_cell(batch_size, rnn_size, num_layers):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    multi_cell = tf.contrib.rnn.MultiRNNCell([lstm]*num_layers)
    initial_state = multi_cell.zero_state(batch_size, tf.float32)
    # to create the cells but with name
    initial_state = tf.identity(initial_state, name="initial_state")
    
    return (multi_cell, initial_state)

In [50]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding_matrix = tf.random_uniform([vocab_size, embed_dim], minval=-1, maxval=1)
    embeded_input = tf.nn.embedding_lookup(embedding_matrix, input_data)
    
    return embeded_input

In [52]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state= tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    # give final_state name
    final_state = tf.identity(final_state, name="final_state")
    
    return (outputs, final_state)

In [53]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embeded_input = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embeded_input)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=tf.nn.relu)
    return (logits, final_state)